In [10]:
file_path = 'web_activity.log'

data = []  # list of dictionaries to store all details

current_ip = None


In [12]:
with open(file_path, 'r') as log_file:
    for line in log_file: 
        line = line.strip()
        
        # If it's an IP line (ends with colon)
        if line.endswith(':'):
            current_ip = line[:-1]  # Remove trailing colon
            continue
        
        # Skip headers or empty lines
        if line == '' or line.startswith('TIME') or line.startswith('STATUS'):
            continue
        
        # Parse the data row
        parts = line.split(None, 3)  # Split into 4 parts: time, method, request, status
        if len(parts) == 4:
            time, method, request, status = parts
            data.append({
                'IP': current_ip,
                'TIME': time,
                'METHOD': method,
                'REQUEST': request,
                'STATUS': status
            })



In [37]:
for entry in data:
    print(f"IP: {entry['IP']}, TIME: {entry['TIME']}, METHOD: {entry['METHOD']}, REQUEST: {entry['REQUEST']}, STATUS: {entry['STATUS']}")

IP: 192.168.0.50, TIME: 2021-06-25T07:23:00.000Z, METHOD: GET, REQUEST: "/", STATUS: 401 (UNAUTHORIZED)
IP: 192.168.0.50, TIME: 2021-06-25T07:23:00.000Z, METHOD: GET, REQUEST: "/login", STATUS: 200 (SUCCESS)
IP: 192.168.0.50, TIME: 2021-06-25T07:23:00.000Z, METHOD: GET, REQUEST: "/login.css", STATUS: 200 (SUCCESS)
IP: 192.168.0.50, TIME: 2021-06-25T07:23:00.000Z, METHOD: GET, REQUEST: "/login.js", STATUS: 200 (SUCCESS)
IP: 192.168.0.50, TIME: 2021-06-25T07:23:44.000Z, METHOD: POST, REQUEST: "/login", STATUS: 200 (SUCCESS)
IP: 192.168.0.50, TIME: 2021-06-25T07:23:45.000Z, METHOD: GET, REQUEST: "/", STATUS: {authorizedUserId: "5Eckr4DTaLLDaDMGqmMJ3g"}                                                   200 (SUCCESS)
IP: 192.168.0.50, TIME: 2021-06-25T07:23:45.000Z, METHOD: GET, REQUEST: "/index.css", STATUS: {authorizedUserId: "5Eckr4DTaLLDaDMGqmMJ3g"}                                          200 (SUCCESS)
IP: 192.168.0.50, TIME: 2021-06-25T07:23:45.000Z, METHOD: GET, REQUEST: "/index.js",

In [29]:
import re
from datetime import datetime
from collections import defaultdict, Counter

print("=== CYBER SECURITY LOG ANALYSIS ===\n")

# 1. CHECK FOR EXTERNAL IPs (Could breach have happened from internet?)
print("1. CHECKING FOR EXTERNAL (NON-PRIVATE) IP ADDRESSES:")
print("-" * 60)

external_ips = []
private_ip_ranges = [
    r'^10\.',           # 10.0.0.0/8
    r'^172\.(1[6-9]|2[0-9]|3[0-1])\.',  # 172.16.0.0/12 #regex expression done using re 
    r'^192\.168\.',     # 192.168.0.0/16
    r'^127\.',          # Loopback
    r'^169\.254\.'      # Link-local
]

for entry in data:
    ip = entry['IP']
    is_private = any(re.match(pattern, ip) for pattern in private_ip_ranges)
    if not is_private:
        external_ips.append(ip)

if external_ips:
    print(f"⚠️  EXTERNAL IPs FOUND: {set(external_ips)}")
    print("❌ BREACH COULD HAVE HAPPENED FROM INTERNET!")
else:
    print("✅ All IPs are internal/private (192.168.x.x range)")
    print("✅ NO EVIDENCE of direct internet breach (attacker was inside network or had VPN)")

print()


=== CYBER SECURITY LOG ANALYSIS ===

1. CHECKING FOR EXTERNAL (NON-PRIVATE) IP ADDRESSES:
------------------------------------------------------------
✅ All IPs are internal/private (192.168.x.x range)
✅ NO EVIDENCE of direct internet breach (attacker was inside network or had VPN)



In [39]:
# 2. EXTRACT USER IDs AND ANALYZE SUSPICIOUS ACTIVITY
print("2. ANALYZING USER ACTIVITY FOR SUSPICIOUS PATTERNS:")
print("-" * 60)

# Extract user IDs from status field
user_sessions = defaultdict(lambda: defaultdict(list))  # {user_id: {ip: [api_requests]}}
ip_sessions = defaultdict(lambda: defaultdict(list))    # {ip: {user_id: [api_requests]}}
api_requests = []

user_id_pattern = re.compile(r'authorizedUserId: "([a-zA-Z0-9]+)"')

for entry in data:
    # Extract user ID if present
    user_match = user_id_pattern.search(entry['STATUS'])
    if user_match:
        user_id = user_match.group(1)
        ip = entry['IP']
        timestamp = datetime.strptime(entry['TIME'], "%Y-%m-%dT%H:%M:%S.000Z")
        
        # Track API requests specifically
        if '/api/' in entry['REQUEST']:
            api_requests.append({
                'user_id': user_id,
                'ip': ip,
                'timestamp': timestamp,
                'request': entry['REQUEST'],
                'method': entry['METHOD']
            })
            user_sessions[user_id][ip].append(timestamp)
            ip_sessions[ip][user_id].append(timestamp)

print(f"Total API requests found: {len(api_requests)}")
print(f"Unique users: {len(user_sessions)}")
print(f"Unique IPs: {len(ip_sessions)}")
print()

2. ANALYZING USER ACTIVITY FOR SUSPICIOUS PATTERNS:
------------------------------------------------------------
Total API requests found: 800
Unique users: 77
Unique IPs: 77



In [40]:
# 3. CHECK FOR USER IDs USED FROM MULTIPLE IPs (Credential sharing/hijacking)
print("3. CHECKING FOR USER IDs USED FROM MULTIPLE IP ADDRESSES:")
print("-" * 60)

suspicious_users = []
for user_id, ips in user_sessions.items():
    if len(ips) > 1:
        suspicious_users.append((user_id, list(ips.keys())))

if suspicious_users:
    print("⚠️  SUSPICIOUS: User IDs used from multiple IPs:")
    for user_id, ips in suspicious_users:
        print(f"   User {user_id}: {ips}")
        # Check if these IPs were used simultaneously
        for ip in ips:
            timestamps = user_sessions[user_id][ip]
            print(f"      {ip}: {len(timestamps)} requests from {min(timestamps)} to {max(timestamps)}")
    print("❌ POTENTIAL credential sharing or session hijacking!")
else:
    print("✅ No user IDs found across multiple IPs")

print()

3. CHECKING FOR USER IDs USED FROM MULTIPLE IP ADDRESSES:
------------------------------------------------------------
✅ No user IDs found across multiple IPs



In [41]:
# 4. DETECT HIGH VOLUME ACTIVITY (Potential scraping/automation)
print("4. CHECKING FOR HIGH VOLUME API ACTIVITY:")
print("-" * 60)

# Count requests per IP and per User
ip_request_counts = Counter(req['ip'] for req in api_requests)
user_request_counts = Counter(req['user_id'] for req in api_requests)

# Set thresholds (adjust based on normal activity patterns)
HIGH_VOLUME_THRESHOLD_IP = 50  # requests per IP
HIGH_VOLUME_THRESHOLD_USER = 50  # requests per user

print("Top 10 IPs by API request volume:")
for ip, count in ip_request_counts.most_common(10):
    status = "⚠️ HIGH" if count > HIGH_VOLUME_THRESHOLD_IP else "✅ Normal"
    print(f"   {ip}: {count} requests - {status}")

print("\nTop 10 Users by API request volume:")
for user_id, count in user_request_counts.most_common(10):
    status = "⚠️ HIGH" if count > HIGH_VOLUME_THRESHOLD_USER else "✅ Normal"
    print(f"   {user_id}: {count} requests - {status}")

print()


4. CHECKING FOR HIGH VOLUME API ACTIVITY:
------------------------------------------------------------
Top 10 IPs by API request volume:
   192.168.0.101: 122 requests - ⚠️ HIGH
   192.168.0.66: 22 requests - ✅ Normal
   192.168.0.54: 20 requests - ✅ Normal
   192.168.0.4: 19 requests - ✅ Normal
   192.168.0.82: 18 requests - ✅ Normal
   192.168.0.59: 17 requests - ✅ Normal
   192.168.0.80: 17 requests - ✅ Normal
   192.168.0.32: 15 requests - ✅ Normal
   192.168.0.63: 15 requests - ✅ Normal
   192.168.0.3: 14 requests - ✅ Normal

Top 10 Users by API request volume:
   mdB7yD2dp1BFZPontHBQ1Z: 122 requests - ⚠️ HIGH
   obbi5BSPBt2K66ZGP8cyTJ: 22 requests - ✅ Normal
   qNFGnHD22ck85MbYPkHA8U: 20 requests - ✅ Normal
   keRUfaP4iwD2ZdrnkeuSBV: 19 requests - ✅ Normal
   dUU1wKpH4atg3s2Qt2fHiP: 18 requests - ✅ Normal
   wCfsVNvQVrEAngWHKDNEHH: 17 requests - ✅ Normal
   wGaAGjLRKsGjvq7QHyDbQc: 17 requests - ✅ Normal
   mTDKsgxXBn64f2yFgPUDMg: 15 requests - ✅ Normal
   4fVRoPdmUWE6oGzGucRyGi: 

In [ ]:
"""high volume 

192.168.0.101: 122 requests - ⚠️ HIGH
mdB7yD2dp1BFZPontHBQ1Z: 122 requests - ⚠️ HIGH

"""

In [43]:
# 5. DETECT REGULAR INTERVAL PATTERNS (Automation/bots)
print("5. CHECKING FOR REGULAR INTERVAL API REQUESTS (AUTOMATION):")
print("-" * 60)

def detect_regular_intervals(timestamps, tolerance_seconds=3):
    """Detect if timestamps show regular intervals (indicating automation)"""
    if len(timestamps) < 3:
        return False, None
    
    # Calculate intervals between consecutive requests
    intervals = []
    for i in range(1, len(timestamps)):
        interval = (timestamps[i] - timestamps[i-1]).total_seconds()
        intervals.append(interval)
    
    if not intervals:
        return False, None
    
    # Check if intervals are regular (within tolerance)
    avg_interval = sum(intervals) / len(intervals)
    regular = all(abs(interval - avg_interval) <= tolerance_seconds for interval in intervals)
    
    return regular, avg_interval

suspicious_automation = []

# Check each user-IP combination for regular patterns
for user_id, ips in user_sessions.items():
    for ip, timestamps in ips.items():
        if len(timestamps) >= 3:  # Need at least 3 requests to detect pattern
            timestamps.sort()  # Ensure chronological order
            is_regular, avg_interval = detect_regular_intervals(timestamps)
            
            if is_regular:
                suspicious_automation.append({
                    'user_id': user_id,
                    'ip': ip,
                    'request_count': len(timestamps),
                    'avg_interval': round(avg_interval, 1),
                    'time_span': timestamps[-1] - timestamps[0]
                })

if suspicious_automation:
    print("⚠️  SUSPICIOUS: Regular interval patterns detected (likely automation):")
    for item in suspicious_automation:
        print(f"   User {item['user_id']} from {item['ip']}:")
        print(f"      {item['request_count']} requests at ~{item['avg_interval']}s intervals")
        print(f"      Time span: {item['time_span']}")
        print()
    print("❌ POTENTIAL automated/bot activity detected!")
else:
    print("✅ No regular interval patterns detected - activity appears manual")

print()


5. CHECKING FOR REGULAR INTERVAL API REQUESTS (AUTOMATION):
------------------------------------------------------------
⚠️  SUSPICIOUS: Regular interval patterns detected (likely automation):
   User irHRppZK35peUiTjxPuCdL from 192.168.0.38:
      3 requests at ~15.5s intervals
      Time span: 0:00:31

❌ POTENTIAL automated/bot activity detected!



In [44]:
# 6. SUMMARY AND RECOMMENDATIONS
print("6. CYBER SECURITY ANALYSIS SUMMARY:")
print("=" * 60)

print("\n📋 FINDINGS:")
print("-" * 20)

# Question 1: Could breach happen from internet?
if external_ips:
    print("🔴 1. INTERNET BREACH: POSSIBLE - External IPs detected")
else:
    print("🟢 1. INTERNET BREACH: UNLIKELY - All activity from internal IPs")

# Question 2: Suspicious activity detected?
suspicious_flags = 0
if suspicious_users:
    print("🔴 2. CREDENTIAL MISUSE: User IDs across multiple IPs detected")
    suspicious_flags += 1

high_volume_ips = [ip for ip, count in ip_request_counts.items() if count > HIGH_VOLUME_THRESHOLD_IP]
high_volume_users = [user for user, count in user_request_counts.items() if count > HIGH_VOLUME_THRESHOLD_USER]

if high_volume_ips or high_volume_users:
    print(f"🟡 3. HIGH VOLUME: {len(high_volume_ips)} IPs and {len(high_volume_users)} users above threshold")
    suspicious_flags += 1

if suspicious_automation:
    print(f"🔴 4. AUTOMATION: {len(suspicious_automation)} regular interval patterns detected")
    suspicious_flags += 1

if suspicious_flags == 0:
    print("🟢 2. SUSPICIOUS ACTIVITY: None detected - activity appears normal")

print(f"\n📊 STATISTICS:")
print(f"   • Total API requests: {len(api_requests)}")
print(f"   • Unique users: {len(user_sessions)}")
print(f"   • Unique IPs: {len(ip_sessions)}")
print(f"   • Suspicious patterns: {suspicious_flags}")

print("\n🎯 CONCLUSIONS:")
print("-" * 20)
if not external_ips and suspicious_flags == 0:
    print("✅ Low risk: Activity appears to be normal internal usage")
elif external_ips:
    print("❌ High risk: External access detected - investigate immediately")
elif suspicious_flags > 0:
    print("⚠️  Medium risk: Suspicious patterns detected - requires investigation")

print("\n" + "=" * 60)


6. CYBER SECURITY ANALYSIS SUMMARY:

📋 FINDINGS:
--------------------
🟢 1. INTERNET BREACH: UNLIKELY - All activity from internal IPs
🟡 3. HIGH VOLUME: 1 IPs and 1 users above threshold
🔴 4. AUTOMATION: 1 regular interval patterns detected

📊 STATISTICS:
   • Total API requests: 800
   • Unique users: 77
   • Unique IPs: 77
   • Suspicious patterns: 2

🎯 CONCLUSIONS:
--------------------
⚠️  Medium risk: Suspicious patterns detected - requires investigation



In [46]:
# DETAILED ANALYSIS OF HIGH-VOLUME ACTIVITY
print("🔍 DETAILED ANALYSIS: IP 192.168.0.101 & User mdB7yD2dp1BFZPontHBQ1Z")
print("=" * 80)

target_ip = "192.168.0.101"
target_user = "mdB7yD2dp1BFZPontHBQ1Z"

# Filter all requests for this IP/User
target_requests = [req for req in api_requests if req['ip'] == target_ip and req['user_id'] == target_user]

print(f"\n📊 BASIC STATISTICS:")
print(f"   • Total API requests: {len(target_requests)}")
print(f"   • Time span: {min(req['timestamp'] for req in target_requests)} to {max(req['timestamp'] for req in target_requests)}")
print(f"   • Duration: {max(req['timestamp'] for req in target_requests) - min(req['timestamp'] for req in target_requests)}")

# Analyze request timing patterns
timestamps = [req['timestamp'] for req in target_requests]
timestamps.sort()

intervals = []
for i in range(1, len(timestamps)):
    interval = (timestamps[i] - timestamps[i-1]).total_seconds()
    intervals.append(interval)

print(f"\n⏱️  TIMING ANALYSIS:")
print(f"   • Average interval: {sum(intervals)/len(intervals):.1f} seconds")
print(f"   • Min interval: {min(intervals):.1f} seconds")
print(f"   • Max interval: {max(intervals):.1f} seconds")
print(f"   • Median interval: {sorted(intervals)[len(intervals)//2]:.1f} seconds")

# Check for automation patterns
consistent_intervals = [i for i in intervals if 9 <= i <= 11]  # 10±1 second intervals
print(f"   • ~10 second intervals: {len(consistent_intervals)}/{len(intervals)} ({len(consistent_intervals)/len(intervals)*100:.1f}%)")

print(f"\n🎯 REQUEST PATTERNS:")


🔍 DETAILED ANALYSIS: IP 192.168.0.101 & User mdB7yD2dp1BFZPontHBQ1Z

📊 BASIC STATISTICS:
   • Total API requests: 122
   • Time span: 2021-06-25 16:14:57 to 2021-06-26 21:00:48
   • Duration: 1 day, 4:45:51

⏱️  TIMING ANALYSIS:
   • Average interval: 855.8 seconds
   • Min interval: 0.0 seconds
   • Max interval: 3600.0 seconds
   • Median interval: 0.0 seconds
   • ~10 second intervals: 0/121 (0.0%)

🎯 REQUEST PATTERNS:


In [47]:
# Analyze what APIs are being called
from collections import Counter
api_endpoints = Counter(req['request'] for req in target_requests)

print("API Endpoints accessed (top 10):")
for endpoint, count in api_endpoints.most_common(10):
    print(f"   {endpoint}: {count} times")

# Extract factory and machine patterns
factory_pattern = re.compile(r'factory=([^&]+)')
machine_pattern = re.compile(r'machine=([^&\s]+)')

factories = []
machines = []

for req in target_requests:
    factory_match = factory_pattern.search(req['request'])
    machine_match = machine_pattern.search(req['request'])
    
    if factory_match:
        factories.append(factory_match.group(1))
    if machine_match:
        machines.append(machine_match.group(1))

factory_counts = Counter(factories)
machine_counts = Counter(machines)

print(f"\n🏭 FACTORY ACCESS PATTERNS:")
for factory, count in factory_counts.most_common():
    print(f"   {factory}: {count} requests")

print(f"\n⚙️  MACHINE ACCESS PATTERNS:")
for machine, count in machine_counts.most_common(10):
    print(f"   {machine}: {count} requests")

print(f"\n🕐 HOURLY ACTIVITY DISTRIBUTION:")
hour_counts = Counter(req['timestamp'].hour for req in target_requests)
for hour in sorted(hour_counts.keys()):
    print(f"   Hour {hour:02d}: {hour_counts[hour]} requests")


API Endpoints accessed (top 10):
   "/api/factory/machine/status?factory=shenzhen&machine=*": 30 times
   "/api/factory/machine/status?factory=meiyo&machine=*": 30 times
   "/api/factory/machine/status?factory=seiko&machine=*": 29 times
   "/api/factory/machine/status?factory=berlin&machine=*": 29 times
   "/api/factory/status?factory=*": 1 times
   "/api/factory/machine/status?factory=shenzhen&machine=Furnace": 1 times
   "/api/factory/machine/status?factory=meiyo&machine=AirWrench": 1 times
   "/api/factory/machine/status?factory=meiyo&machine=HeavyDutyDrill": 1 times

🏭 FACTORY ACCESS PATTERNS:
   meiyo: 32 requests
   shenzhen: 31 requests
   seiko: 29 requests
   berlin: 29 requests
   *": 1 requests

⚙️  MACHINE ACCESS PATTERNS:
   *": 118 requests
   Furnace": 1 requests
   AirWrench": 1 requests
   HeavyDutyDrill": 1 requests

🕐 HOURLY ACTIVITY DISTRIBUTION:
   Hour 00: 4 requests
   Hour 01: 4 requests
   Hour 02: 4 requests
   Hour 03: 4 requests
   Hour 04: 4 requests
   Hou

In [48]:
# Check for automation indicators
print(f"\n🤖 AUTOMATION ANALYSIS:")

# 1. Regular timing patterns
regular_10s = len([i for i in intervals if 9.5 <= i <= 10.5])
regular_30s = len([i for i in intervals if 29 <= i <= 31])
regular_60s = len([i for i in intervals if 59 <= i <= 61])

print(f"   • ~10 second intervals: {regular_10s}/{len(intervals)} ({regular_10s/len(intervals)*100:.1f}%)")
print(f"   • ~30 second intervals: {regular_30s}/{len(intervals)} ({regular_30s/len(intervals)*100:.1f}%)")
print(f"   • ~60 second intervals: {regular_60s}/{len(intervals)} ({regular_60s/len(intervals)*100:.1f}%)")

# 2. Repetitive request patterns
request_sequences = []
for i in range(len(target_requests)-2):
    seq = (target_requests[i]['request'], target_requests[i+1]['request'], target_requests[i+2]['request'])
    request_sequences.append(seq)

sequence_counts = Counter(request_sequences)
repeated_sequences = [seq for seq, count in sequence_counts.items() if count > 1]

print(f"   • Repeated 3-request sequences: {len(repeated_sequences)}")
if repeated_sequences:
    print("     Top repeated patterns:")
    for seq, count in sequence_counts.most_common(3):
        if count > 1:
            print(f"       {count}x: {' → '.join(seq[:2])} → ...")

# 3. Check for exact timing patterns (bot-like behavior)
exact_intervals = Counter(intervals)
common_intervals = [interval for interval, count in exact_intervals.items() if count >= 5]

print(f"   • Exact intervals repeated 5+ times: {len(common_intervals)}")
for interval in sorted(common_intervals):
    print(f"     {interval}s: {exact_intervals[interval]} times")

print(f"\n🚨 RISK ASSESSMENT:")
risk_factors = []

if regular_10s/len(intervals) > 0.5:
    risk_factors.append("High percentage of 10-second intervals")
if len(repeated_sequences) > 5:
    risk_factors.append("Multiple repeated request sequences")
if len(common_intervals) > 2:
    risk_factors.append("Multiple exact timing patterns")
if len(target_requests) > 100:
    risk_factors.append("Extremely high request volume")

if risk_factors:
    print("❌ HIGH RISK - Likely automated/bot activity:")
    for factor in risk_factors:
        print(f"   • {factor}")
elif len(target_requests) > 50:
    print("⚠️  MEDIUM RISK - Unusual activity patterns")
else:
    print("✅ LOW RISK - Activity appears manual")

print("\n" + "=" * 80)



🤖 AUTOMATION ANALYSIS:
   • ~10 second intervals: 0/121 (0.0%)
   • ~30 second intervals: 0/121 (0.0%)
   • ~60 second intervals: 0/121 (0.0%)
   • Repeated 3-request sequences: 4
     Top repeated patterns:
       29x: "/api/factory/machine/status?factory=meiyo&machine=*" → "/api/factory/machine/status?factory=seiko&machine=*" → ...
       29x: "/api/factory/machine/status?factory=seiko&machine=*" → "/api/factory/machine/status?factory=shenzhen&machine=*" → ...
       28x: "/api/factory/machine/status?factory=shenzhen&machine=*" → "/api/factory/machine/status?factory=berlin&machine=*" → ...
   • Exact intervals repeated 5+ times: 2
     0.0s: 87 times
     3600.0s: 28 times

🚨 RISK ASSESSMENT:
❌ HIGH RISK - Likely automated/bot activity:
   • Extremely high request volume



In [49]:
# BRUTE FORCE LOGIN ATTEMPT ANALYSIS
print("🔍 BRUTE FORCE LOGIN ATTEMPT ANALYSIS")
print("=" * 80)

# Filter unauthorized access attempts
unauthorized_attempts = [entry for entry in data if entry['STATUS'] == '401 (UNAUTHORIZED)']

# Group by IP and count attempts
from collections import defaultdict
ip_attempts = defaultdict(list)

for attempt in unauthorized_attempts:
    ip_attempts[attempt['IP']].append(attempt['TIME'])

# Identify IPs with frequent unauthorized attempts
suspicious_ips = {ip: times for ip, times in ip_attempts.items() if len(times) > 5}

print(f"\n📊 SUSPICIOUS IPs WITH FREQUENT UNAUTHORIZED ATTEMPTS:")
for ip, times in suspicious_ips.items():
    print(f"   {ip}: {len(times)} attempts")
    # Check for rapid attempts
    times.sort()
    intervals = [(datetime.strptime(times[i+1], "%Y-%m-%dT%H:%M:%S.000Z") - datetime.strptime(times[i], "%Y-%m-%dT%H:%M:%S.000Z")).total_seconds() for i in range(len(times)-1)]
    rapid_attempts = [interval for interval in intervals if interval < 60]
    print(f"   Rapid attempts (<60s apart): {len(rapid_attempts)}")

print("\n🚨 RISK ASSESSMENT:")
if suspicious_ips:
    print("❌ HIGH RISK - Potential brute force detected:")
    for ip in suspicious_ips:
        print(f"   • {ip}")
else:
    print("✅ LOW RISK - No significant brute force patterns detected")

print("\n" + "=" * 80)

🔍 BRUTE FORCE LOGIN ATTEMPT ANALYSIS

📊 SUSPICIOUS IPs WITH FREQUENT UNAUTHORIZED ATTEMPTS:

🚨 RISK ASSESSMENT:
✅ LOW RISK - No significant brute force patterns detected

